<a href="https://colab.research.google.com/github/Alfrermn/g20_p1_aprendizaje_automatico/blob/a/modelo_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MODELO FINAL

EJERCICIO 8

c)

In [6]:
# Primeramente leemos los datos y los guardamos en dos variables distintas, disp_df y comp_df, con los datos de entrenamiento y pruebas respectivamente.
import pandas as pd
disp_df = pd.read_csv("/disp_st20ns1.txt.bz2",
 compression="bz2",
 index_col=0)
comp_df = pd.read_csv("/comp_st20ns1.txt.bz2",
 compression="bz2",
 index_col=0)

In [7]:
import pickle

#Evaluación de hiperparámetros ajustados para regresión lineal
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import time


X = disp_df.drop(['salida'], axis=1)
y = disp_df['salida']

# Dividimos los datos en conjuntos de entrenamiento y tests
X_train, X_test, y_train, y_test= train_test_split(X, y , shuffle=False, test_size=0.16) 


#Separamos para el predefined split, con 10 años de entrenamiento y dos de tests
n_train = int(10*365)
n_valid = int(2*365)
selector = [-1] * n_train + [0] * n_valid
split = PredefinedSplit(selector)

lr = LinearRegression()

param = {
'positive': [True, False],
'n_jobs': [-1, 0, 1],
'fit_intercept': [True, False],
'copy_X': [True, False]
}

grid_lr = GridSearchCV(estimator=lr, param_grid=param, scoring='neg_mean_squared_error', cv=split)
grid_lr.fit(X, y)

#Obtenemos los mejores hiperparámetros y su puntuación
best_params = grid_lr.best_params_
best_score = grid_lr.best_score_

#Entrenamos el modelo con los mejores hiperparámetros en el conjunto de entrenamiento
best_lr = LinearRegression(**best_params)
start = time.time()
best_lr.fit(X_train, y_train)
end = time.time()

#Realizammos predicciones en el conjunto de prueba
y_pred = best_lr.predict(X_test)

#Calculamos la puntuación del modelo en el conjunto de prueba
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)

print("Mejores hiperparámetros encontrados: ", best_params)
print("Puntuación RMSE en conjunto de prueba: ", rmse)
print("El Tiempo de Entrenamiento es: ", (end-start))

# Guardar modelo en archivo
with open('modelo_final.pkl', 'wb') as f:
    pickle.dump(lr, f)


Mejores hiperparámetros encontrados:  {'copy_X': True, 'fit_intercept': False, 'n_jobs': -1, 'positive': False}
Puntuación RMSE en conjunto de prueba:  3275311.065951866
El Tiempo de Entrenamiento es:  0.04644966125488281
